In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Layer
import numpy as np

In [2]:
class IdentityBlock(tf.keras.Model):
    def __init__(self, filters=32,kernal_size=3):
        super(IdentityBlock, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters, kernal_size, padding='same')
        self.batch_norm = tf.keras.layers.BatchNormalization()
        
        self.conv2 = tf.keras.layers.Conv2D(filters, kernal_size, padding='same')
        self.add = tf.keras.layers.Add()
        
        self.activation = tf.keras.layers.Activation('relu')
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.batch_norm(x)
        
        x = self.activation(x)
        x = self.conv2(x)
        x = self.batch_norm(x)
        x = self.add([x, inputs])
        x = self.activation(x)
        return x

In [3]:
class MiniResnet(tf.keras.Model):
    def __init__(self, num_of_classes):
        super(MiniResnet, self).__init__()
        self.conv = tf.keras.layers.Conv2D(64,7, padding="same", input_shape=(28,28,1))
        self.norm = tf.keras.layers.BatchNormalization()
        self.act = tf.keras.layers.Activation('relu')
        self.max_pool = tf.keras.layers.MaxPool2D((3, 3))
        
        self.block1 = IdentityBlock(64, 3)
        self.block2 = IdentityBlock(64, 3)
        
        self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.classifier = tf.keras.layers.Dense(num_of_classes, activation="softmax")
    
    def call(self, inputs):
        x = self.conv(inputs)
        x = self.norm(x)
        x = self.act(x)
        x = self.max_pool(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.global_pool(x)
        x = self.classifier(x)
        return x
        

In [4]:
mini_resnet = MiniResnet(10)
mini_resnet.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test)= mnist.load_data()

In [6]:
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [7]:
mini_resnet.fit(np.expand_dims(np.array(x_train),-1), np.array(y_train), 
                validation_data=(np.expand_dims(np.array(x_test),-1), np.array(y_test)), 
                epochs=2, batch_size=32)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 207s 3ms/sample - loss: 0.1359 - accuracy: 0.9644 - val_loss: 4.7000 - val_accuracy: 0.3133
Epoch 2/2
60000/60000 [==============================] - 203s 3ms/sample - loss: 0.0506 - accuracy: 0.9847 - val_loss: 2.4312 - val_accuracy: 0.4094
